<a href="https://colab.research.google.com/github/FrancoPalavicinoG/cellia/blob/main/notebooks/09_baseline_training_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load Train, Val & Test Datatsets from Google Drive

Mount Drive

In [70]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Import libraries

In [71]:
# Core
import pandas as pd
import numpy as np

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier

# Metrics
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, classification_report

# Explainability
import shap
import matplotlib.pyplot as plt
import seaborn as sns

Load Datasets

In [72]:
input_path = "/content/drive/MyDrive/cellia_drive/Datasets/"

Load features

In [73]:
X_train = pd.read_csv(input_path + "X_train_cat.csv")
X_val   = pd.read_csv(input_path + "X_val_cat.csv")
X_test  = pd.read_csv(input_path + "X_test_cat.csv")

Load labels

In [74]:
y_train = pd.read_csv(input_path + "y_train_cat.csv").squeeze()
y_val   = pd.read_csv(input_path + "y_val_cat.csv").squeeze()
y_test  = pd.read_csv(input_path + "y_test_cat.csv").squeeze()

In [75]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(897, 26) (897,)
(782, 26) (782,)
(783, 26) (783,)


### Evaluation function

In [76]:
def evaluate_model(model, X, y_true, dataset_name="Validation"):
    y_pred = model.predict(X)
    y_proba = model.predict_proba(X)[:, 1]

    roc_auc = roc_auc_score(y_true, y_proba)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)

    print(f"{dataset_name} Results:")
    print(f"ROC-AUC: {roc_auc:.3f} | F1: {f1:.3f} | Precision: {precision:.3f} | Recall: {recall:.3f}")
    print("-" * 60)

    return {"roc_auc": roc_auc, "f1": f1, "precision": precision, "recall": recall}

## Model Training

### Logistic Regression (Baseline)

In [77]:
log_reg = LogisticRegression(
    max_iter=500,          # iteration numbers
    random_state=42,
    class_weight='balanced',  # class balancing
    C=1.0,                 # regularization
    penalty='l2',          # penalty type: 'l1', 'l2', 'elasticnet'
    solver='liblinear'     # solver compatible with the penalty
)


### Random Forest

In [78]:
rf = RandomForestClassifier(
    n_estimators=300,      # number of trees
    max_depth=10,          # maximum depth of each tree
    min_samples_split=2,   # minimum samples to split a node
    min_samples_leaf=1,    # minimum samples per leaf
    max_features='sqrt',   # number of features to consider per split
    random_state=42,
    class_weight='balanced'
)


### XGBoost

In [79]:
xgb = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric='logloss'
)

### LightGBM

In [80]:
lgbm = LGBMClassifier(
    n_estimators=300,      # number of iterations
    max_depth=6,           # maximum tree depth
    num_leaves=31,         # leaves per tree
    learning_rate=0.05,    # learning rate
    min_child_samples=20,  # minimum samples per leaf node
    subsample=0.8,         # fraction of data per tree
    colsample_bytree=0.8,  # fraction of features per tree
    random_state=42,
    scale_pos_weight=1     # class balancing if not using SMOTE
)

### MLP Tabular

In [81]:
mlp = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=300, random_state=42)

### Train models

In [82]:
models = {"Logistic Regression": log_reg,
          "Random Forest": rf,
          "LightGBM": lgbm,
          "MLP": mlp,
          "XGBoost": xgb}

results = {}

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    results[name] = evaluate_model(model, X_val, y_val, f"Validation ({name})")

Training Logistic Regression...
Validation (Logistic Regression) Results:
ROC-AUC: 0.844 | F1: 0.395 | Precision: 0.264 | Recall: 0.779
------------------------------------------------------------
Training Random Forest...
Validation (Random Forest) Results:
ROC-AUC: 0.841 | F1: 0.390 | Precision: 0.272 | Recall: 0.688
------------------------------------------------------------
Training LightGBM...
[LightGBM] [Info] Number of positive: 359, number of negative: 538
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000191 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1445
[LightGBM] [Info] Number of data points in the train set: 897, number of used features: 26
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.400223 -> initscore=-0.404536
[LightGBM] [Info] Start training from score -0.404536
[LightGBM] [Warning] No further splits with p

/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


Validation (MLP) Results:
ROC-AUC: 0.781 | F1: 0.329 | Precision: 0.220 | Recall: 0.649
------------------------------------------------------------
Training XGBoost...
Validation (XGBoost) Results:
ROC-AUC: 0.848 | F1: 0.427 | Precision: 0.303 | Recall: 0.727
------------------------------------------------------------


### Compare models

In [83]:
models = {"Logistic Regression": log_reg,
          "Random Forest": rf,
          "LightGBM": lgbm,
          "MLP": mlp,
          "XGBoost": xgb}

results = {}
for name, model in models.items():
    results[name] = evaluate_model(model, X_val, y_val, f"Validation ({name})")

results_df = pd.DataFrame(results).T
results_df.sort_values("roc_auc", ascending=False)

Validation (Logistic Regression) Results:
ROC-AUC: 0.844 | F1: 0.395 | Precision: 0.264 | Recall: 0.779
------------------------------------------------------------
Validation (Random Forest) Results:
ROC-AUC: 0.841 | F1: 0.390 | Precision: 0.272 | Recall: 0.688
------------------------------------------------------------
Validation (LightGBM) Results:
ROC-AUC: 0.839 | F1: 0.418 | Precision: 0.293 | Recall: 0.727
------------------------------------------------------------
Validation (MLP) Results:
ROC-AUC: 0.781 | F1: 0.329 | Precision: 0.220 | Recall: 0.649
------------------------------------------------------------
Validation (XGBoost) Results:
ROC-AUC: 0.848 | F1: 0.427 | Precision: 0.303 | Recall: 0.727
------------------------------------------------------------


,roc_auc,f1,precision,recall
XGBoost,0.848430,0.427481,0.302703,0.727273
Logistic Regression,0.844174,0.394737,0.264317,0.779221
Random Forest,0.840729,0.389706,0.271795,0.688312
LightGBM,0.839329,0.417910,0.293194,0.727273
MLP,0.781100,0.328947,0.220264,0.649351


## Hyperparameter Tuning using RandomizedSearchCV

In [84]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from scipy.stats import uniform, randint
import numpy as np

# ---------------------------
# Logistic Regression
# ---------------------------
logreg = LogisticRegression(max_iter=1000, random_state=42)

logreg_param_dist = {
    'C': uniform(0.01, 10),
    'penalty': ['l2'],
    'solver': ['liblinear', 'saga'],
    'class_weight': [None, 'balanced']
}

logreg_search = RandomizedSearchCV(
    logreg,
    param_distributions=logreg_param_dist,
    n_iter=30,
    scoring='f1',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

logreg_search.fit(X_train, y_train)
print("Best Logistic Regression params:", logreg_search.best_params_)
print("Best F1:", logreg_search.best_score_)

# ---------------------------
# Random Forest
# ---------------------------
rf = RandomForestClassifier(random_state=42)

rf_param_dist = {
    'n_estimators': randint(200, 1000),
    'max_depth': randint(5, 20),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5),
    'max_features': ['sqrt', 'log2', None],
    'class_weight': [None, 'balanced', 'balanced_subsample']
}

rf_search = RandomizedSearchCV(
    rf,
    param_distributions=rf_param_dist,
    n_iter=10,
    scoring='f1',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

rf_search.fit(X_train, y_train)
print("Best Random Forest params:", rf_search.best_params_)
print("Best F1:", rf_search.best_score_)

# ---------------------------
# LightGBM
# ---------------------------
lgbm = LGBMClassifier(random_state=42)

lgbm_param_dist = {
    'num_leaves': randint(20, 50),
    'max_depth': randint(5, 15),
    'learning_rate': uniform(0.01, 0.1),
    'n_estimators': randint(200, 1000),
    'min_child_samples': randint(10, 50),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'class_weight': [None, 'balanced']
}

lgbm_search = RandomizedSearchCV(
    lgbm,
    param_distributions=lgbm_param_dist,
    n_iter=10,
    scoring='f1',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

lgbm_search.fit(X_train, y_train)
print("Best LightGBM params:", lgbm_search.best_params_)
print("Best F1:", lgbm_search.best_score_)


# ---------------------------
# XGBoost
# ---------------------------
xgb = XGBClassifier(
    random_state=42,
    eval_metric='logloss',
    use_label_encoder=False
)

xgb_param_dist = {
    'n_estimators': randint(200, 800),
    'learning_rate': uniform(0.01, 0.2),
    'max_depth': randint(3, 10),
    'min_child_weight': randint(1, 10),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'gamma': uniform(0, 5),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 2),
    'scale_pos_weight': [1, (y_train.value_counts()[0] / y_train.value_counts()[1])]
}

xgb_search = RandomizedSearchCV(
    xgb,
    param_distributions=xgb_param_dist,
    n_iter=20,
    scoring='f1',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

xgb_search.fit(X_train, y_train)
print("Best XGBoost params:", xgb_search.best_params_)
print("Best F1:", xgb_search.best_score_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best Logistic Regression params: {'C': np.float64(0.35388521115218396), 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'liblinear'}
Best F1: 0.7295678679909625
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Random Forest params: {'class_weight': 'balanced_subsample', 'max_depth': 15, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 9, 'n_estimators': 863}
Best F1: 0.7269109880794972
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[LightGBM] [Info] Number of positive: 359, number of negative: 538
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000175 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1445
[LightGBM] [Info] Number of data points in the train set: 897, number of used features: 26
[LightGBM] [Info] [binary:B

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [13:37:50] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


## Optimize Hyperparameters Models

### Get the best pre-trained models directly from RandomizedSearchCV.

In [85]:
best_logreg = logreg_search.best_estimator_
best_rf = rf_search.best_estimator_
best_lgbm = lgbm_search.best_estimator_
best_xgb = xgb_search.best_estimator_

from sklearn.metrics import classification_report, f1_score

models = {
    "Logistic Regression": best_logreg,
    "Random Forest": best_rf,
    "LightGBM": best_lgbm,
    "XGBoost": best_xgb
}

results = {}
for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f"\n=== {name} ===")
    print("F1 Score:", f1_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    results[name] = evaluate_model(model, X_val, y_val, f"Validation ({name})")

results_df = pd.DataFrame(results).T
results_df.sort_values("roc_auc", ascending=False)


=== Logistic Regression ===
F1 Score: 0.3860759493670886
              precision    recall  f1-score   support

           0       0.97      0.75      0.84       706
           1       0.26      0.79      0.39        77

    accuracy                           0.75       783
   macro avg       0.61      0.77      0.62       783
weighted avg       0.90      0.75      0.80       783

Validation (Logistic Regression) Results:
ROC-AUC: 0.845 | F1: 0.393 | Precision: 0.263 | Recall: 0.779
------------------------------------------------------------

=== Random Forest ===
F1 Score: 0.3917525773195876
              precision    recall  f1-score   support

           0       0.96      0.78      0.86       706
           1       0.27      0.74      0.39        77

    accuracy                           0.77       783
   macro avg       0.62      0.76      0.63       783
weighted avg       0.90      0.77      0.82       783

Validation (Random Forest) Results:
ROC-AUC: 0.844 | F1: 0.377 | Precis

,roc_auc,f1,precision,recall
XGBoost,0.852593,0.391304,0.257143,0.818182
LightGBM,0.851266,0.412371,0.280374,0.779221
Logistic Regression,0.845316,0.393443,0.263158,0.779221
Random Forest,0.843622,0.377104,0.254545,0.727273


### Export trained models

In [ ]:
import joblib

output_path = "/content/drive/MyDrive/cellia_drive/Models/"

import os
os.makedirs(output_path, exist_ok=True)

# Save each model
for name, model in models.items():
    file_path = os.path.join(output_path, f"{name}.joblib")
    joblib.dump(model, file_path)
    print(f"✅ {name} saved to {file_path}")